In [351]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import re

from bs4 import BeautifulSoup
from utils import *
from row_utils import *

# Set directory & name of save file
results = []
save_file_name = "others_df"

path_2004 = "./2005/"
all_2004 = os.listdir(path_2004)
all_2004 = [x for x in all_2004 if ('DS_Store' not in x)]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [352]:
# Recording which files have been scraped 
cik_done_dict = get_cik_done(csv_name)

In [353]:
cik_done_dict

{65984: [20041231, 20041231, 20041231, 20041231]}

In [356]:
# Scraping the remaining files
for i, fname in enumerate(all_2004):
    test_f = open(path_2004 + fname)
    wholetext = test_f.read()
    cikbase = re.search(r"CENTRAL INDEX KEY:\s*?\d{10}\b", wholetext, re.I).group()
    cik_re = re.split(r'\s', cikbase)[-1]        

    datadatebase = re.search(r"CONFORMED PERIOD OF REPORT:\s*?\d{8}\b", wholetext, re.I).group() 
    datadate_re = re.split(r'\s', datadatebase)[-1] 
    if int(cik_re) in cik_done_dict:
        if int(datadate_re) in cik_done_dict[int(cik_re)]:
            print("Already processed this file")
            continue
    
    wholetext = wholetext.lower()
    tbl_flag = 0 # 1 if "contractual obligations" table exists
    row_flag = 0 # 1 if "purchasing obligations" row exists
    multiplier = 1 # 1 by default
    
    table = scrape_table(wholetext)
    rows = []
    if table:
        tbl_flag = 1
        rows = scrape_rows(table)
        multiplier = get_multiplier_from_tbl(table)
    else:
        lines, tbl_list = scrape_text_table(wholetext)
        if tbl_list:
            tbl_flag = 1
            rows = scrape_text_rows(lines, tbl_list)
            multiplier = get_multiplier_from_tbl_list(tbl_list)

            
    for row in rows:
        row_flag = 1
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 'row': row_flag, 
                   '<1': row['<1'], '1-3':row['1-3'], '3-5':row['3-5'], '>5':row['>5'], 
                  'category': row['category'], 'multiplier': multiplier}
        if 'total' in row.keys():
                result['total'] = row['total']
        results.append(result)

    # No table or no row
    if (tbl_flag == 0) or (row_flag == 0):
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 'row': row_flag, 
                  'total': '-', '<1': '-', '1-3':'-', '3-5':'-', '>5':'-', 'multiplier': multiplier}

        results.append(result)
    pd.DataFrame(results).to_csv(save_file_name)
    print("File: ", i, "\tTable: ", tbl_flag, "Row: ", row_flag)


Already processed this file
File:  1 	Table:  1 Row:  1
File:  2 	Table:  1 Row:  1
File:  3 	Table:  1 Row:  1
File:  4 	Table:  1 Row:  1
File:  5 	Table:  1 Row:  1
File:  6 	Table:  1 Row:  1
File:  7 	Table:  1 Row:  0
File:  8 	Table:  1 Row:  1
File:  9 	Table:  1 Row:  1
File:  10 	Table:  1 Row:  1
File:  11 	Table:  1 Row:  1
File:  12 	Table:  1 Row:  1
File:  13 	Table:  1 Row:  1
File:  14 	Table:  1 Row:  1
File:  15 	Table:  1 Row:  1
File:  16 	Table:  1 Row:  1
File:  17 	Table:  1 Row:  1
File:  18 	Table:  1 Row:  1
File:  19 	Table:  1 Row:  1
File:  20 	Table:  1 Row:  1
File:  21 	Table:  1 Row:  1
File:  22 	Table:  1 Row:  1
File:  23 	Table:  1 Row:  1
File:  24 	Table:  1 Row:  1
File:  25 	Table:  1 Row:  1
File:  26 	Table:  1 Row:  1
File:  27 	Table:  1 Row:  1
File:  28 	Table:  1 Row:  1
File:  29 	Table:  1 Row:  1
File:  30 	Table:  1 Row:  0
File:  31 	Table:  1 Row:  1
File:  32 	Table:  1 Row:  1
File:  33 	Table:  1 Row:  0
File:  34 	Table:  1 Row

In [349]:
#results_df = pd.DataFrame(results)
results_df = pd.read_csv('others_df')
row_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 1)] # table but no row
none_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 0)] # no table no row
last_check = results_df[(results_df['row'] == 1) & (results_df['tbl'] == 1)] # table and row, other issue?
print("row & table found: ", len(set(last_check['fname'])))
print("table found: ", len(set(row_check['fname'])))
print("nothing found: ", len(set(none_check['fname'])))

row & table found:  1
table found:  0
nothing found:  0


In [348]:
last_check

,Unnamed: 0,1-3,3-5,<1,>5,category,cik,datadate,fname,multiplier,row,tbl,total
0,0,10,6,7.0,11,operating leases,65984,20041231,0000065984-05-000147.txt,1000000,1,1,34.0
1,1,361,336,190.0,2059,purchase obligations (2),65984,20041231,0000065984-05-000147.txt,1000000,1,1,2946.0
2,2,10,6,7.0,11,operating leases,65984,20041231,0000065984-05-000147.txt,1000000,1,1,34.0
3,3,361,336,190.0,2059,purchase obligations (2),65984,20041231,0000065984-05-000147.txt,1000000,1,1,2946.0


In [350]:
cik_done_dict

{4962: [20021231],
 4977: [20021231, 20021231, 20021231],
 5187: [20021231],
 6207: [20030430],
 6383: [20030228, 20030228],
 6885: [20030201, 20030201],
 6951: [20021027],
 7084: [20030630],
 7214: [20021231],
 7536: [20021231, 20021231],
 7623: [20021130],
 7694: [20030930],
 7789: [20021231],
 8411: [20030930],
 8598: [20021231],
 8670: [20030912],
 9128: [20021231],
 9342: [20021228],
 9346: [20021231],
 9389: [20021231],
 9548: [20021231],
 9779: [20030630, 20030630, 20030630],
 9892: [20021231, 20021231, 20021231, 20021231, 20021231, 20021231],
 10427: [20021228, 20021228],
 11860: [20021231],
 12208: [20021231],
 12355: [20021231],
 12400: [20021231],
 12779: [20021031],
 12927: [20021231, 20021231, 20021231],
 13239: [20021231],
 13372: [20021231],
 13573: [20030629],
 14280: [20031014],
 14693: [20030430],
 14707: [20030201],
 14846: [20030930],
 14995: [20030930, 20020930],
 15310: [20021231],
 15393: [20021231],
 15486: [20021228, 20021228, 20021228],
 15615: [20021231],
 15

# Examining one file

In [316]:
path_prefix = "./2005/"
test_f = open(path_prefix + "0001144204-05-009084.txt")
wholetext = test_f.read()
cik_re = get_cik(wholetext)
datadate_re = get_datadate(wholetext)  
print("CIK: ", cik_re, "Date: ", datadate_re)
wholetext = wholetext.lower()
#lines, tbl_list = scrape_text_table(wholetext)
lines, tbl_list = scrape_text_table(wholetext)
rows = scrape_text_rows(lines, tbl_list)
#scores, tables = scrape_table(wholetext)
#table = scrape_table(wholetext)
#rows = scrape_rows(table)
#lines
rows

CIK:  0000096988 Date:  20041231
['contractual obligations total', '1', 'year years years years']
['commitment for this facility is', '214000', 'through september', '22', '2007', 'which is']
['store fixtures will be essentially unchanged in', '2005', 'as compared with', '2004.']
['the company wrote off accounts receivable and inventories in', '2001', 'as a result of']
ROWS
[['long0term debt obligations', '5057', '46', '34', '4977', '00'], ['capital lease obligations', '96', '38', '58', '00', '00'], ['operating lease obligations', '7427', '2100', '4601', '726', '00']]
total_idx:  -1
['capital lease obligations', '96', '38', '58', '00', '00']


[{'1-3': 34.0,
  '3-5': '4977',
  '<1': 46.0,
  '>5': '00',
  'category': 'long0term debt obligations',
  'total': 5057.0},
 {'1-3': 58.0,
  '3-5': '00',
  '<1': 38.0,
  '>5': '00',
  'category': 'capital lease obligations',
  'total': 96.0},
 {'1-3': 4601.0,
  '3-5': '726',
  '<1': 2100.0,
  '>5': '00',
  'category': 'operating lease obligations',
  'total': 7427.0}]

In [209]:
#table_to_list(table)
table_to_list(tables[np.where(scores == 4)[0][1]])
#scores

IndexError: index 1 is out of bounds for axis 0 with size 1

In [293]:
#lines = chunks[np.argmax(scores)].split('\n')
lines = [x for x in lines if (hasNumbers(x))]
keywords = HDR_KWS + ROW_KWS + ['obligations']
lines = [x for x in lines if (hasKeywords(x, keywords))]
lines = [x.split(' ') for x in lines]
for i, line in enumerate(lines):
    lines[i] = list(filter(lambda a: (a != '') and (a != '$') and (a !='<u>') and (a !='</u>'), line))    
tbl_list = []
tbl_list = combine_text_fields(lines)
scrape_text_rows(lines, tbl_list)

Unknown error:  could not convert string to float: 'by inflation of'


[{'1-3': 2848000.0,
  '3-5': '71000',
  '<1': 1179000.0,
  '>5': '000',
  'category': 'obligations',
  'total': 4098000.0},
 {'1-3': 24293.0,
  '3-5': '0',
  '<1': 24293.0,
  '>5': '0',
  'category': 'obligations',
  'total': 48586.0},
 {'1-3': 0.0,
  '3-5': '0',
  '<1': 3450.0,
  '>5': '0',
  'category': 'purchase obligations',
  'total': 3450.0}]

In [135]:
#lt[1].text.replace('\n', '').replace('\xa0', '')
len(lt)

1